In [ ]:
from importlib import import_module
import pandas as pd
import numpy as np

from vantage6.algorithm.tools.util import info
from vantage6.algorithm.tools.exceptions import AlgorithmExecutionError

In [ ]:
# Temporary disable some privacy settings that are defined in the v6-summary-py
_summary = import_module("v6-summary-py")
_summary.utils.DEFAULT_MINIMUM_ROWS = 0
_summary.utils.DEFAULT_PRIVACY_THRESHOLD = 0

_summary.partial_summary.DEFAULT_MINIMUM_ROWS = 0
_summary.partial_summary.DEFAULT_PRIVACY_THRESHOLD = 0

In [ ]:
# Read test data
df1 = pd.read_csv('/Users/vramella/Library/CloudStorage/OneDrive-BIOMERIS/PROGETTI-CLIENTI/RC-INT/KWF - Blueberry/Vantage6/Algoritmi-dev/summary-mod//data_org1.csv')
df2 = pd.read_csv('/Users/vramella/Library/CloudStorage/OneDrive-BIOMERIS/PROGETTI-CLIENTI/RC-INT/KWF - Blueberry/Vantage6/Algoritmi-dev/summary-mod//data_org2.csv')

In [ ]:
def split_into_cohorts(df, cohort_1_fraction=0.6):
    mask = np.random.rand(len(df)) < cohort_1_fraction
    cohort_1 = df[mask].copy()
    cohort_2 = df[~mask].copy()
    return cohort_1, cohort_2

In [ ]:
# Organization 1
sarcoma_cohort_1, sarcoma_cohort_2 = split_into_cohorts(df1)
org1 = {"cohort_1": sarcoma_cohort_1, "cohort_2": sarcoma_cohort_2}

In [ ]:
# Organization 2
sarcoma_cohort_1, sarcoma_cohort_2 = split_into_cohorts(df2)
org2 = {"cohort_1": sarcoma_cohort_1, "cohort_2": sarcoma_cohort_2}

In [ ]:
# Define columns to compute the summary statistics for
columns = ['Sex', 'age', 'Height', 'Weight']
numeric_columns = ['age', 'Height', 'Weight']

In [ ]:
def _aggregate_partial_summaries(results: list[dict]) -> dict:
    """Aggregate the partial summaries of all nodes.

    Parameters
    ----------
    results : list[dict]
        The partial summaries of all nodes.
    """
    info("Aggregating partial summaries")
    aggregated_summary = {}
    is_first = True
    for result in results:
        if result is None:
            raise AlgorithmExecutionError(
                "At least one of the nodes returned invalid result. Please check the "
                "logs."
            )
        if is_first:
            # copy results. Only convert num complete rows per node to a list so that
            # we can add the other nodes to it later
            aggregated_summary = result
            aggregated_summary["num_complete_rows_per_node"] = [
                result["num_complete_rows_per_node"]
            ]
            for column in result["numeric"]:
                aggregated_summary["numeric"][column]["median"] = [
                    result["numeric"][column]["median"]
                ]
                aggregated_summary["numeric"][column]["q_25"] = [
                    result["numeric"][column]["q_25"]
                ]
                aggregated_summary["numeric"][column]["q_75"] = [
                    result["numeric"][column]["q_75"]
                ]
            is_first = False
            continue

        # aggregate data for numeric columns
        for column in result["numeric"]:
            aggregated_dict = aggregated_summary["numeric"][column]
            aggregated_dict["count"] += result["numeric"][column]["count"]
            aggregated_dict["min"] = min(
                aggregated_summary["numeric"][column]["min"],
                result["numeric"][column]["min"],
            )
            aggregated_dict["max"] = max(
                aggregated_summary["numeric"][column]["max"],
                result["numeric"][column]["max"],
            )
            aggregated_dict["missing"] += result["numeric"][column]["missing"]
            aggregated_dict["sum"] += result["numeric"][column]["sum"]
            aggregated_dict["median"].append(result["numeric"][column]["median"])
            aggregated_dict["q_25"].append(result["numeric"][column]["q_25"])
            aggregated_dict["q_75"].append(result["numeric"][column]["q_75"])

        # aggregate data for categorical columns
        for column in result["categorical"]:
            aggregated_dict = aggregated_summary["categorical"][column]
            aggregated_dict["count"] += result["categorical"][column]["count"]
            aggregated_dict["missing"] += result["categorical"][column]["missing"]

        # add the number of complete rows for this node
        aggregated_summary["num_complete_rows_per_node"].append(
            result["num_complete_rows_per_node"]
        )

        # add the unique values
        for column in result["counts_unique_values"]:
            if column not in aggregated_summary["counts_unique_values"]:
                aggregated_summary["counts_unique_values"][column] = {}
            for value, count in result["counts_unique_values"][column].items():
                if value not in aggregated_summary["counts_unique_values"][column]:
                    aggregated_summary["counts_unique_values"][column][value] = 0
                aggregated_summary["counts_unique_values"][column][value] += count

    # now that all data is aggregated, we can compute the mean
    for column in aggregated_summary["numeric"]:
        aggregated_dict = aggregated_summary["numeric"][column]
        if aggregated_dict["count"]:
            aggregated_dict["mean"] = aggregated_dict["sum"] / aggregated_dict["count"]
        else:
            aggregated_dict["mean"] = 0  # TODO this is terrible, we should not do this

    return aggregated_summary

In [ ]:
def _add_sd_to_results(
    results: dict, variance_results: list[dict], numerical_columns: list[str]
) -> dict:
    """Add the variance to the results.

    Parameters
    ----------
    results : dict
        The results of the summary task.
    variance_results : list[dict]
        The variance results of all nodes.
    numerical_columns : list[str]
        The numerical columns.

    Returns
    -------
    dict
        The results with the variance added.
    """
    for column in numerical_columns:
        sum_variance = 0
        for node_results in variance_results:
            sum_variance += node_results[column]
        if results["numeric"][column]["count"] > 1:
            variance = sum_variance / (results["numeric"][column]["count"] - 1)
        else:
            variance = 0  # TODO THIS IS TERRIBLE
        results["numeric"][column]["std"] = variance**0.5
    return results

In [ ]:
def summary_per_data_station(
    dfs: list[pd.DataFrame], cohort_names: list[str], *args, **kwargs
) -> dict:
    results = {}
    for df, name in zip(dfs, cohort_names):
        results[name] = _summary.partial_summary._summary_per_data_station(
            df, *args, **kwargs
        )
        # Add median and quantiles (0.25, 0.75)
        for var in results[name]['numeric']:
            results[name]['numeric'][var]['median'] = float(np.nanmedian(df[var]))
            results[name]['numeric'][var]['q_25'] = float(np.nanquantile(df[var], 0.25))
            results[name]['numeric'][var]['q_75'] = float(np.nanquantile(df[var], 0.75))

    return results

In [ ]:
def variance_per_data_station(
    dfs: list[pd.DataFrame],
    cohort_names: list[str],
    means: dict[list[float]],
    *args,
    **kwargs,
) -> dict:
    results = {}
    info(kwargs)
    info(means)
    for df, name in zip(dfs, cohort_names):
        info("*" * 80)
        info(name)
        info(means[name])
        info("*" * 80)
        results[name] = _summary.partial_variance._variance_per_data_station(
            df, means=means[name], *args, **kwargs
        )
    return results

In [ ]:
res1 = summary_per_data_station(dfs=list(org1.values()), cohort_names=list(org1.keys()), columns=columns, numeric_columns=numeric_columns)

In [ ]:
res2 = summary_per_data_station(dfs=list(org2.values()), cohort_names=list(org2.keys()), columns=columns, numeric_columns=numeric_columns)

In [ ]:
results = [res1, res2]
print(results)

In [ ]:
# aggregate the partial summaries of all nodes
all_cohort_results = {}

means = {}
cohort_names = results[0].keys()
print(cohort_names)

In [ ]:
for cohort_name in cohort_names:
    print(cohort_name)
    for result in results:
        print(result[cohort_name])
    cohort_results = [result[cohort_name] for result in results]
    all_cohort_results[cohort_name] = _aggregate_partial_summaries(cohort_results)

    numerical_columns = list(all_cohort_results[cohort_name]["numeric"].keys())
    # compute the variance now that we have the mean
    means[cohort_name] = [
        all_cohort_results[cohort_name]["numeric"][column]["mean"]
        for column in numerical_columns
    ]
    info(f"n num cols: {len(numerical_columns)}")
    info(f"n means: {len(means[cohort_name])}")

    info("debugger")
    info(numerical_columns)
    info(len(means))

In [ ]:
all_cohort_results

In [ ]:
# Variance per data station
var1 = variance_per_data_station(dfs=list(org1.values()), cohort_names=list(org1.keys()), means=means, columns=numerical_columns)
var2 = variance_per_data_station(dfs=list(org2.values()), cohort_names=list(org2.keys()), means=means, columns=numerical_columns)
variance_results = [var1, var2]

In [ ]:
variance_results

In [ ]:
for cohort_name in cohort_names:
        cohort_variance_results = [result[cohort_name] for result in variance_results]
        all_cohort_results[cohort_name] = _add_sd_to_results(
            all_cohort_results[cohort_name], cohort_variance_results, numerical_columns
        )

In [ ]:
all_cohort_results